# Show Commands Script

Up to this point Netmiko has been used to push configuration data and validation returned in nice to read human format.

In this lab we are going to utilize Network2Code's NTC templates using Google's TextFSM and Cisco's Genie library to return structured data.

### Import Statements

In [1]:
import yaml
from getpass import getpass
from netmiko import Netmiko
from pprint import pprint

### Read Inventory

In [2]:
def read_inventory(inventory_file):
    with open(inventory_file, 'r') as file:
        hosts = yaml.safe_load(file)
        return hosts

### Display output using Genie

Genie is a Cisco project.

```
pyATS and Genie were developed side by side in the same team, in proper software stack/layers, with pyATS being the generic all-purpose framework, and Genie extending its capabilities and specializing in network device automation & validation.
```

To utilize Genie within Netmiko there are a couple of pip dependancies, and the container already has them installed.
```bash
pip install genie
pip install pyats
```

To view all the available parsers that genie supports please visit [https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/parsers](https://pubhub.devnetcloud.com/media/genie-feature-browser/docs/#/parsers)

Once all the dependancies have been installed to utilize Genie all you have to do is set ```use_genie=True``` within send_command.

In [3]:
def show_commands_genie(session, commands):
    for command in commands:
        print(f"Now running: {command}")
        output = session.send_command(command, use_genie=True)
        pprint(output) 
        pprint(type(output))

### Display output using TextFSM

TextFSM is a Python project by Google:
```
TextFSM is a template based state machine to parse semi-formatted text into tables. It was developed by Google and is hosted on Google Code. You find also various forks on GitHub, e.g. for python 3 support (which we will later use). It is highly customizable, because it works with separate template definitions, which contains variables and rules with regular expressions. This library is very helpful to parse any text-based CLI output from network devices. TextFSM can be used within python scripts or as a standalone tool. I think you have now an idea, what TextFSM is. Lets start with the example.
```

Network To Code has utilized Google's TextFSM project and created templates for parsing Cisco and other vendor's show commands. 

It does require some setup to run correctly, and if you are using this from the container the work has already been completed. 

If you wish to the view the templates they are stored in 
```bash
/ntc-templates/templates
```
The library and installation instructions can be found at: ```https://github.com/networktocode/ntc-templates```

Once all the dependancies have been installed to utilize TextFSM all you have to do is set ```use_textfsm=True``` within send_command.

In [4]:
def show_commands_textfsm(session, commands):
    for command in commands:
        print(f"Now running: {command}")
        output = session.send_command(command, use_textfsm=True)
        pprint(output) 
        pprint(type(output))

### Main Section

In [5]:
if __name__ == '__main__':
    inventory_file = 'inventory/routers.yml'
    # List of show commands
    commands = ['show ip interface brief']

In [6]:
    # Prompt for username and password
    username = input('Enter the username: ')
    password = getpass('Enter the password: ')
    # Read in the inventory
    hosts = read_inventory(inventory_file)
    
    # Loop through the inventory and use a context handler for the Netmiko session
    for key, value in hosts.items():
        device = {'host': value['host'], 'port': value['port'],
                  'username': username, 'password': password,
                  'device_type': value['device_type']}
        with Netmiko(**device) as session:
            print(f"Now runing for {key}")
            # Uncomment to display output using Genie
            print('Using Genie')
            show_commands_genie(session, commands)
            print('-'*100)
            print('Using TextFSM')
            show_commands_textfsm(session, commands)          
            print('-'*100) 
            print('-'*100)

Enter the username: admin
Enter the password: ········
Now runing for csr1
Using Genie
Now running: show ip interface brief
{'interface': {'GigabitEthernet1': {'interface_is_ok': 'YES',
                                    'ip_address': '172.16.1.10',
                                    'method': 'NVRAM',
                                    'protocol': 'up',
                                    'status': 'up'},
               'GigabitEthernet2': {'interface_is_ok': 'YES',
                                    'ip_address': '192.168.100.1',
                                    'method': 'NVRAM',
                                    'protocol': 'up',
                                    'status': 'up'},
               'GigabitEthernet3': {'interface_is_ok': 'YES',
                                    'ip_address': 'unassigned',
                                    'method': 'NVRAM',
                                    'protocol': 'down',
                                    'status': 'administrati